In [70]:
import pandas as pd
import numpy as np
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In [7]:
files = [f for f in os.listdir('.') if os.path.isfile(f)]
for f in files:
    print(f)

Gainesville_background.geojson
Gainesville_BG.gpkg


In [84]:
df = pd.read_csv('../Tabular/Gainesville_Zillowdata.csv')

gdf = gpd.GeoDataFrame(
    df, 
    geometry=geopandas.points_from_xy(df['long'], df['lat']), 
    crs="EPSG:4326"
)

gdf = gdf.drop(columns=['Unnamed: 0', 'index', 'index_right'])


block_groups = gpd.read_file('Gainesville_BG.gpkg')
block_groups.crs = gdf.crs


joined_df = gdf.sjoin(block_groups, how='left', predicate='within')

joined_df = joined_df[['zpid', 'address', 'zestimate', 'zestimate_30_DayDelta', 'GEOID']]

zillow_bg = joined_df.groupby('GEOID').agg({
    'zestimate':'median',
    'zestimate_30_DayDelta':['median','count']
}).reset_index()

zillow_bg.columns = ["_".join(a) for a in zillow_bg.columns.to_flat_index()]

zillow_bg['zestimate_30_DayDelta_count'] = np.where(zillow_bg['zestimate_30_DayDelta_count'] < 10, 'Insufficient data', zillow_bg['zestimate_30_DayDelta_count'])


zillow_bg.rename(columns={
    'GEOID_':'GEOID',
    'zestimate_30_DayDelta_median':'median_30_change'}, 
                 inplace=True)

zillow_bg['GEOID'] = zillow_bg['GEOID'].astype(int)


zillow_bg.dtypes

GEOID                            int64
zestimate_median               float64
median_30_change               float64
zestimate_30_DayDelta_count     object
dtype: object

In [91]:
df = pd.read_csv('../Gainesville_test.csv')

df = df.merge(zillow_bg, left_on='Block Group', right_on='GEOID')

df = df.drop(columns=['Unnamed: 0', 'GEOID', 'zestimate_30_DayDelta_count'])

# df.to_csv('../Gainesville_test2.csv')

df.head()

,Block Group,HH_below Poverty_Value,Percentage of 2020 Total Households (ACS 5-Yr),Percentage of 2020 Total Population,2022 Baby Boomer Pop,Value_x,Percentage of 2022 Total Population_x,2022 Generation Alpha Pop,2022 Generation X Pop,2022 Generation Z Pop,...,2022 Population Age 5-9,2022 Total Population,Value_y.1,Percentage of 2022 Total Population,2027 Population Age 10-14,2027 Population Age 15-19,2027 Population Age 5-9,2027 Total Population,zestimate_median,median_30_change
0,131390003071,13,1.72%,77.35%,539,29,1.50%,123,343,388,...,115,"1,931","1,476",76.44%,128,107,124,"1,948",798450.0,1.571861
1,131390004011,0,0.00%,56.75%,297,64,4.41%,121,294,334,...,109,"1,451",806,55.55%,119,105,110,"1,498",334900.0,1.944751
2,131390004012,54,7.22%,67.86%,426,376,17.23%,180,388,474,...,124,"2,182","1,460",66.91%,118,109,120,"2,181",578250.0,1.503936
3,131390004021,0,0.00%,83.58%,390,31,1.84%,114,302,331,...,101,"1,682","1,393",82.82%,110,93,106,"1,724",532900.0,1.961880
4,131390005012,50,7.55%,50.11%,237,254,14.57%,220,234,515,...,183,"1,743",854,49.00%,180,116,177,"1,728",327400.0,1.990950
